In [1]:
from seemps.state import MPS
import torch

n = 3
N = 2**n
v = torch.rand(N)
v_mps = MPS.from_vector(v, [2]*n, normalize=False)

print(v)
print(v_mps.to_vector())

tensor([0.1171, 0.8760, 0.4644, 0.7956, 0.7782, 0.3334, 0.5310, 0.0073])
tensor([0.1171, 0.8760, 0.4644, 0.7956, 0.7782, 0.3334, 0.5310, 0.0073])


In [2]:
import torch
from seemps.state import MPS 
from seemps.operators import MPO, MPOSum, MPOList
from seemps.truncate.simplify_mpo import simplify_mpo
from seemps.truncate.simplify import SIMPLIFICATION_STRATEGY

strategy = SIMPLIFICATION_STRATEGY.replace(normalize=False)

I = torch.eye(2, dtype=torch.cfloat)
X = torch.tensor([[0, 1], [1, 0]], dtype=torch.cfloat)
Y = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.cfloat)
Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.cfloat)

def kron_all(op_list):
    result = op_list[0]
    for op in op_list[1:]:
        result = torch.kron(result, op)
    return result
    
def lift_to_full(op, site, size, I):
    ops = [I] * size
    ops[site] = op
    return kron_all(ops)

def op_to_tensor_list(op, site, size, I):
    ops = [I.reshape(1,2,2,1)] * size
    ops[site] = op.reshape(1,2,2,1)
    return ops

def forward(size):
    I = torch.eye(2, dtype=torch.complex64)
    X = torch.tensor([[0, 1], [1, 0]], dtype=torch.complex64)
    Y = torch.tensor([[0, -1j], [1j, 0]], dtype=torch.complex64)
    Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64)
    herm = [I, X, Y, Z]
    
    alpha = torch.rand(size, len(herm), dtype=torch.complex64)  # real coefficients
    
    psi_terms = []
    psi_tensors = []
    mpos = []
    mpos_exp = []
    
    for i in range(size):
        # Local operator on site i
        psi_i = sum(alpha[i, j] * herm[j] for j in range(len(herm)))
        
        # Lift to full Hilbert space
        psi_lift = lift_to_full(psi_i, i, size, I)
        
        # Exponential of psi_i using torch.linalg.matrix_exp
        psi_exp = torch.linalg.matrix_exp(psi_i)
        
        mpos.append(MPO(op_to_tensor_list(psi_i, i, size, I)))
        mpos_exp.append(MPO(op_to_tensor_list(psi_exp, i, size, I)))
        psi_terms.append(psi_lift)
        psi_tensors.append(psi_i.reshape(1,2,2,1))
    
    return sum(psi_terms), MPOSum(mpos).join(), MPOList(mpos_exp).join()

n = 5
N = 2**n
v = torch.rand(N, dtype=torch.complex64)
v_mps = MPS.from_vector(v.numpy(), [2]*n, normalize=False)  # if MPS requires numpy, convert here
print(v)
print(v_mps.to_vector())
print('Initial state is equal is', torch.allclose(v, v_mps.to_vector()))
M, M_mpo, M_exp_mpo = forward(n)
M_mpo = simplify_mpo(M_mpo, strategy=strategy)
r1 = M @ v
r2 = M_mpo @ v_mps
print(r1)
print(r2.to_vector())
print('Final state is equal is', torch.allclose(r1, r2.to_vector()))


tensor([0.4374+0.8461j, 0.7180+0.7931j, 0.8691+0.7630j, 0.4358+0.7467j,
        0.8764+0.5144j, 0.4689+0.2281j, 0.6172+0.9755j, 0.7818+0.8571j,
        0.3599+0.4794j, 0.1468+0.0906j, 0.1499+0.3426j, 0.3434+0.9103j,
        0.8442+0.6833j, 0.3862+0.4584j, 0.3463+0.4100j, 0.9084+0.0181j,
        0.7613+0.2427j, 0.4962+0.7860j, 0.0532+0.0171j, 0.6890+0.8659j,
        0.4040+0.6046j, 0.2528+0.0181j, 0.3049+0.3731j, 0.9533+0.0288j,
        0.5205+0.9575j, 0.3895+0.9914j, 0.4225+0.1174j, 0.4539+0.3395j,
        0.9426+0.5799j, 0.7081+0.5067j, 0.4595+0.1947j, 0.5193+0.4055j])
tensor([0.4374+0.8461j, 0.7180+0.7931j, 0.8691+0.7630j, 0.4358+0.7467j,
        0.8764+0.5144j, 0.4689+0.2281j, 0.6172+0.9755j, 0.7818+0.8571j,
        0.3599+0.4794j, 0.1468+0.0906j, 0.1499+0.3426j, 0.3434+0.9103j,
        0.8442+0.6833j, 0.3862+0.4584j, 0.3463+0.4100j, 0.9084+0.0181j,
        0.7613+0.2427j, 0.4962+0.7860j, 0.0532+0.0171j, 0.6890+0.8659j,
        0.4040+0.6046j, 0.2528+0.0181j, 0.3049+0.3731j, 0.9533+

In [1]:
import torch

# Example: why it works
complex_matrix = torch.randn(4, 4, dtype=torch.complex64, requires_grad=True)

# Your original approach - FAILS
try:
    U, S, V = torch.svd(complex_matrix)
    loss = (S**2).sum()
    loss.backward()  # This crashes!
except RuntimeError as e:
    print(f"Original fails: {e}")

# Our approach - WORKS
def complex_svd_safe(matrix):
    real_part = matrix.real
    imag_part = matrix.imag
    
    # Block matrix construction
    top = torch.cat([real_part, -imag_part], dim=-1)
    bottom = torch.cat([imag_part, real_part], dim=-1)
    real_matrix = torch.cat([top, bottom], dim=-2)
    
    # SVD on real matrix (gradients work!)
    U_real, S_real, V_real = torch.svd(real_matrix)
    
    # Reconstruct complex results
    m, n = matrix.shape[-2:]
    U_top = U_real[..., :m, :n]
    U_bottom = U_real[..., m:, :n]
    U_complex = U_top + 1j * U_bottom
    
    V_left = V_real[..., :n, :n]
    V_right = V_real[..., n:, :n]
    V_complex = V_left + 1j * V_right
    
    return U_complex, S_real[..., :n], V_complex.conj().transpose(-2, -1)

# This works!
U, S, V = complex_svd_safe(complex_matrix)
loss = (S**2).sum()
loss.backward()  # No error!
print("Complex SVD safe: SUCCESS!")

Complex SVD safe: SUCCESS!


In [ ]:
M_mpo = simplify_mpo(M_mpo, strategy=strategy)
print([m.shape for m in M_mpo])
M_exp_mpo = simplify_mpo(M_exp_mpo, strategy=strategy)
print([m.shape for m in M_exp_mpo])

r1 = M @ v
r2 = M_mpo @ v_mps
print(torch.allclose(r1, torch.tensor(r2.to_vector())))

M_exp = torch.linalg.matrix_exp(M)
r1 = M_exp @ v
r2 = M_exp_mpo @ v_mps
print(torch.allclose(r1, torch.tensor(r2.to_vector())))


In [89]:
import torch
import numpy as np 
from seemps.state import MPS 
from seemps.operators import MPO, MPOSum, MPOList
from seemps.truncate.simplify_mpo import simplify_mpo
from seemps.truncate.simplify import SIMPLIFICATION_STRATEGY
import scipy

strategy = SIMPLIFICATION_STRATEGY.replace(normalize=False)

I = torch.tensor([[1, 0],[0, 1]])
X = torch.tensor([[0, 1],[1, 0]])
Y = torch.tensor([[0, -1j],[1j, 0]])
Z = torch.tensor([[1, 0],[0, -1]])
import tensorkrowch as tk
def input_to_mps(x, dims):
    mps_tensors = tk.decompositions.vec_to_mps(x.reshape(*dims),
                                                   n_batches=0,
                                                   cum_percentage=1,
                                                   renormalize=False)
    x_tk = tk.models.MPSData(n_features=len(dims),
                                            phys_dim=dims,
                                            bond_dim=1,
                                            n_batches=0,
                                            boundary='obc')
    x_tk.add_data(mps_tensors)
    return x_tk
def kron_all(op_list):
    result = op_list[0]
    for op in op_list[1:]:
        result = np.kron(result, op)
    return result
    
def lift_to_full(op, site, size, I):
    ops = [I] * size
    ops[site] = op
    return kron_all(ops)

def op_to_tensor_list(op, site, size, I):
    ops = [I.reshape(1,2,2,1)] * size
    ops[site] = op.reshape(1,2,2,1)
    return ops
def forward(size):
    """
    Constructs the psi and phi matrices from the alpha and beta parameters.
    This defines the forward pass of the discriminator model.
    """
    herm = [I,X,Y,Z]
    alpha = torch.rand(size, len(herm))

    # Build lifted local ops and sum them
    psi_terms = []
    psi_tensors = []
    mpos = []
    mpos_exp = []
    for i in range(size):
        # Local operator on site i
        psi_i = sum(alpha[i, j] * herm[j] for j in range(len(herm)))

        # Lift to full Hilbert space
        psi_lift = lift_to_full(psi_i, i, size, I)
        tensor_list = mpo_seemps_to_tk(op_to_tensor_list(psi_i, i, size, I))
        mpos.append(MPO(op_to_tensor_list(psi_i, i, size, I)))
        mpos_exp.append(MPO(op_to_tensor_list(scipy.linalg.expm(psi_i), i, size, I)))
        psi_terms.append(psi_lift)
        psi_tensors.append(psi_i.reshape(1,2,2,1))
    return sum(psi_terms), MPOSum(mpos).join(), MPOList(mpos_exp).join()
n = 2
N = 2**n
v = torch.rand(N)
v_tk = input_to_mps(v, dims=[2]*n)
print('Original state is', torch.allclose(v, mps2vector(v_tk)))

v_mps = MPS.from_vector(v, [2]*n, normalize=False)
M, M_mpo, M_exp_mpo = forward(n)
print([m.shape for m in M_mpo])
M_exp_mpo = simplify_mpo(M_exp_mpo, strategy=strategy)
print([m.shape for m in M_exp_mpo])
r1 = M @ v
r2 = M_mpo @ v_mps
print(np.allclose(r1,r2.to_vector()))
M_exp = scipy.linalg.expm(M)
r1 = M_exp @ v
r2 = M_exp_mpo @ v_mps
print(np.allclose(r1,r2.to_vector()))


n = 2
N = 2**n
v = np.random.rand(N)
v_tk = input_to_mps(torch.from_numpy(v), dims=[2]*n)
print(v)
print(mps2vector(v_tk).numpy())

Original state is True
[(1, 2, 2, 2), (2, 2, 2, 1)]
[(1, 2, 2, 1), (1, 2, 2, 1)]


TypeError: unsupported operand type(s) for @: 'numpy.ndarray' and 'Tensor'

In [83]:
import tensorkrowch as tk
def input_to_mps(x, dims):
    mps_tensors = tk.decompositions.vec_to_mps(x.reshape(*dims),
                                                   n_batches=0,
                                                   cum_percentage=1,
                                                   renormalize=False)
    x_tk = tk.models.MPSData(n_features=len(dims),
                                            phys_dim=dims,
                                            bond_dim=1,
                                            n_batches=0,
                                            boundary='obc')
    x_tk.add_data(mps_tensors)
    return x_tk

In [67]:
from seemps.state import scprod

print(scprod((r2+1j*r2).join(),r2), np.vdot(r1+1j*r1,r1))

(1905.6496927363687-1905.6496927363685j) (1905.6496927363883-1905.6496927363883j)


In [58]:
n = 2
N = 2**n
v = np.random.rand(N)
v_mps = MPS.from_vector(v, [2]*n, normalize=False)
M, M_mpo, M_exp_mpo = forward(n)
M_mpo = simplify_mpo(M_mpo, strategy=strategy)
print([m.shape for m in M_mpo])
M_exp_mpo = simplify_mpo(M_exp_mpo, strategy=strategy)
print([m.shape for m in M_exp_mpo])
r1 = M @ v
r2 = M_mpo @ v_mps
print(np.allclose(r1,r2.to_vector()))
M_exp = scipy.linalg.expm(M)
r1 = M_exp @ v
r2 = M_exp_mpo @ v_mps
print(np.allclose(r1,r2.to_vector()))

[(1, 2, 2, 2), (2, 2, 2, 1)]
[(1, 2, 2, 1), (1, 2, 2, 1)]
True
True


In [59]:
print(r1)
print(r2.to_vector())

[9.02198101-1.6398981j  5.30899805+1.58449381j 4.34645306+0.08948784j
 2.23216227+1.22174247j]
[9.02198101-1.6398981j  5.30899805+1.58449381j 4.34645306+0.08948784j
 2.23216227+1.22174247j]


In [68]:

n = 2
N = 2**n
v = np.random.rand(N)
op = np.kron(X, I)
op_v = op @ v
v_mps = MPS.from_vector(v, [2]*n, normalize=False)
op_mpo = MPO([X.reshape(1,2,2,1), I.reshape(1,2,2,1)])
op_v_mps = op_mpo @ v_mps
print(op_v)
print(op_v_mps.to_vector())

[0.87783222 0.17682039 0.77328938 0.58367155]
[0.87783222 0.17682039 0.77328938 0.58367155]


In [70]:
from mps import * 
n = 2
N = 2**n
v = np.random.rand(N)
op = np.kron(X, I)
op_v = op @ v
v_mps = MPS.from_vector(v, [2]*n, normalize=False)
op_mpo = MPO([X.reshape(1,2,2,1), I.reshape(1,2,2,1)])
op_v_mps = op_mpo @ v_mps
print(op_v)
print(op_v_mps.to_vector())

[0.18738808 0.32541538 0.50923964 0.64179717]
[0.18738808 0.32541538 0.50923964 0.64179717]


In [5]:
n = 2
N = 2**n
v = np.random.rand(N)
op = np.kron(X, I)
op_v = op @ v
v_mps = MPS.from_vector(v, [2]*n, normalize=False)
op_mpo = MPO([X.reshape(1,2,2,1), I.reshape(1,2,2,1)])
op_v_mps = op_mpo @ v_mps
print(op_v)
print(op_v_mps.to_vector())

[0.48533172 0.72145719 0.39140277 0.08542669]
[0.48533172 0.72145719 0.39140277 0.08542669]
